# Binance Liquidation Stream Websocket (Futures) V2.0

### Overall, this script efficiently collects, processes, and saves liquidation order data from the Binance WebSocket stream, ensuring real-time monitoring and data persistence

In [ ]:
import websocket
import pandas as pd
import json
import threading
import schedule
import time

df_liquid = pd.DataFrame(columns=['Symbol', 'Order_Type', 'Price', 'Original_Quantity', "last_filled_quantity", "Side",'Order_Status'])
ws = None  # Initialize ws as a global variable
save_interval = 1800  # Save interval in seconds (1 hour)

def save_and_clear_df():
    global df_liquid  # Declare df_liquid as global
    while True:
        time.sleep(save_interval)  # Wait for the specified interval
        current_time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
        # Convert relevant columns to numeric types
        df_liquid['Price'] = pd.to_numeric(df_liquid['Price'])
        df_liquid['Original_Quantity'] = pd.to_numeric(df_liquid['Original_Quantity'])
        df_liquid['last_filled_quantity'] = pd.to_numeric(df_liquid['last_filled_quantity'])
        # Calculate derived columns
        df_liquid['original_volume'] = df_liquid['Price'] * df_liquid['Original_Quantity']
        df_liquid['last_filled_volume'] = df_liquid['Price'] * df_liquid['last_filled_quantity']
        # Save DataFrame to CSV
        df_liquid.to_csv(f"Liquidation-{current_time}.csv")
        print(f"(Liquidation) Chapter Saved at {current_time}")
        # Clear DataFrame by creating a new empty DataFrame
        df_liquid = pd.DataFrame(columns=['Symbol', 'Order_Type', 'Price', 'Original_Quantity', "last_filled_quantity", "Side", 'Order_Status'])
        print(f"(Liquidation) Clearing Dataframe at {current_time}")

# Create a thread to save and clear DataFrame
save_thread = threading.Thread(target=save_and_clear_df)
save_thread.daemon = True  # Set the thread as daemon so it terminates when the main thread exits
save_thread.start()

def on_message(ws, message):
    msg = json.loads(message)
    if msg["e"] == "forceOrder":
        stream_liquidation(msg)

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### Closed ###")

def on_open(ws):
    print("### Opened ###")

def stream_liquidation(msg):
    # Extract required items from the message
    event_time = pd.to_datetime(msg["E"], unit='ms')  # Convert timestamp to datetime
    symbol = msg["o"]["s"]
    side = msg["o"]["S"]
    order_type = msg["o"]["o"]
    time_in_force = msg["o"]["f"]
    original_quantity = msg["o"]["q"]
    price = msg["o"]["p"]
    average_price = msg["o"]["ap"]
    order_status = msg["o"]["X"]
    last_filled_quantity = msg["o"]["l"]
    filled_accumulated_quantity = msg["o"]["z"]
    order_trade_time = msg["o"]["T"]

    # Print out the liquidation order information
    print(f"LIQUIDATION: Event Time: {event_time} | Symbol: {symbol} | Side: {side} | Order Type: {order_type} | Price: {price} | Original  Quantity: {original_quantity} | Last Filled  Quantity: {last_filled_quantity} | Order Status: {order_status}", end="\r")
    df_liquid.loc[event_time] = [symbol, order_type, price, original_quantity, last_filled_quantity, side, order_status]


# Define the WebSocket URL for Liquidation Order Streams
socket_url = "wss://fstream.binance.com/ws/!forceOrder@arr"

# Create and run the WebSocket connection in a separate thread
def run_websocket():
    websocket.enableTrace(False)
    global ws
    ws = websocket.WebSocketApp(socket_url, on_message=on_message, on_error=on_error, on_close=on_close)
    ws.on_open = on_open
    ws.run_forever()

def stop_websocket():
    global ws  # Access the global ws variable
    if ws:
        ws.close()  # Close the WebSocket connection if it's open

websocket_thread = threading.Thread(target=run_websocket)
websocket_thread.start()


In [ ]:
stop_websocket()